PIDNet needs GTs about boundaries too, and we can extract them from the masks themselves using cv2.Canny (at least that's how the author of the model does it)

In [1]:
from google.colab import drive
import os

SAVE_ON_DRIVE = False
drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/LoveDA/Train'
if (os.path.exists("./Train") == False):
    if (os.path.exists("/content/drive/MyDrive/LoveDA/Train.zip")):
        print("Dataset available on own drive, unzipping...")
        !unzip -q /content/drive/MyDrive/LoveDA/Train.zip -d ./
    else:
        print("Downloading dataset...")
        !wget -O Train.zip "https://zenodo.org/records/5706578/files/Train.zip?download=1"
        if(SAVE_ON_DRIVE):
            print("Saving dataset on drive...")
            !cp Train.zip /content/drive/MyDrive/LoveDA/
        !unzip -q Train.zip -d ./

else:
    print("Dataset already in local")

Mounted at /content/drive
--2025-01-25 15:55:10--  https://zenodo.org/records/5706578/files/Train.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4021669263 (3.7G) [application/octet-stream]
Saving to: ‘Train.zip’

Train.zip           100%[===================>]   3.75G  24.0MB/s    in 2m 9s   

2025-01-25 15:57:19 (29.7 MB/s) - ‘Train.zip’ saved [4021669263/4021669263]



In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
import os
from tqdm import tqdm

rural_masks_path = "./Train/Rural/masks_png"
urban_masks_path = "./Train/Urban/masks_png"

rural_boundaries_path = "./Train/Rural/boundaries_png"
urban_boundaries_path = "./Train/Urban/boundaries_png"

domains = ["URBAN", "RURAL"]

for domain in domains:
  if domain == "URBAN":
      masks_path = urban_masks_path
      boundaries_path = urban_boundaries_path
  elif domain == "RURAL":
      masks_path = rural_masks_path
      boundaries_path = rural_boundaries_path
  else:
      raise ValueError(f"Invalid domain, should be either URBAN or RURAL, instead is {domain}")

  if (os.path.exists(boundaries_path) == False):
      print(f"Creating {boundaries_path}...")
      os.makedirs(boundaries_path)
  else:
      print(f"{boundaries_path} exists...")

  loop = tqdm(enumerate(os.listdir(masks_path)))

  for i, filename in loop:
      mask = cv2.imread(os.path.join(masks_path, filename), cv2.IMREAD_GRAYSCALE)
      edge = cv2.Canny(mask, 0.1, 0.2)
      edge_before_dilation = edge.copy()

      kernel = np.ones((4, 4), np.uint8)
      edge = edge[6:-6, 6:-6]
      edge = np.pad(edge, ((6,6),(6,6)), mode='constant')
      edge = (cv2.dilate(edge, kernel, iterations=1)>50)*1.0

      # Save
      cv2.imwrite(os.path.join(boundaries_path, filename), edge)


      # # Display using matplotlib.pyplot
      # plt.figure(figsize=(10, 5))

      # plt.subplot(1, 3, 1)
      # plt.imshow(mask, cmap='gray')
      # plt.title('Mask')

      # plt.subplot(1, 3, 3)
      # plt.imshow(edge_before_dilation, cmap='gray')
      # plt.title('Edges Before Dilation')

      # plt.subplot(1, 3, 2)
      # plt.imshow(edge, cmap='gray')
      # plt.title('Edges')

      # plt.show()

Creating ./Train/Urban/boundaries_png...


1156it [00:20, 57.52it/s]


Creating ./Train/Rural/boundaries_png...


1366it [00:23, 57.78it/s]


In [3]:
for i, row in enumerate(edge):
    for j, col in enumerate(row):
        if col != 0 and col != 1:
            print(f"Found {col} at {i},{j}")

# Shouldn't have values in between, make sure to have only either 0s or 1s

In [4]:
import os

# Define the directory path
rural_boundaries_path = './Train/Rural/boundaries_png'
rural_images_path = './Train/Rural/images_png'
rural_masks_path = './Train/Rural/masks_png'

urban_boundaries_path = './Train/Urban/boundaries_png'
urban_images_path = './Train/Urban/images_png'
urban_masks_path = './Train/Urban/masks_png'


# Count the number of files in the directory
rural_bd_file_count = len([name for name in os.listdir(rural_boundaries_path) if os.path.isfile(os.path.join(rural_boundaries_path, name))])
rural_img_file_count = len([name for name in os.listdir(rural_images_path) if os.path.isfile(os.path.join(rural_images_path, name))])
rural_mask_file_count = len([name for name in os.listdir(rural_masks_path) if os.path.isfile(os.path.join(rural_masks_path, name))])

urban_bd_file_count = len([name for name in os.listdir(urban_boundaries_path) if os.path.isfile(os.path.join(urban_boundaries_path, name))])
urban_img_file_count = len([name for name in os.listdir(urban_images_path) if os.path.isfile(os.path.join(urban_images_path, name))])
urban_mask_file_count = len([name for name in os.listdir(urban_masks_path) if os.path.isfile(os.path.join(urban_masks_path, name))])

print(f'There are {rural_bd_file_count} files in {rural_boundaries_path}')
print(f'There are {rural_img_file_count} files in {rural_images_path}')
print(f'There are {rural_mask_file_count} files in {rural_masks_path}')

print(f'There are {urban_bd_file_count} files in {urban_boundaries_path}')
print(f'There are {urban_img_file_count} files in {urban_images_path}')
print(f'There are {urban_mask_file_count} files in {urban_masks_path}')

There are 1366 files in ./Train/Rural/boundaries_png
There are 1366 files in ./Train/Rural/images_png
There are 1366 files in ./Train/Rural/masks_png
There are 1156 files in ./Train/Urban/boundaries_png
There are 1156 files in ./Train/Urban/images_png
There are 1156 files in ./Train/Urban/masks_png


In [5]:
import shutil
import os

rural_boundaries_path = './Train/Rural/boundaries_png'
urban_boundaries_path = './Train/Urban/boundaries_png'

drive_rural_boundaries_path = '/content/drive/MyDrive/LoveDA/boundaries/Rural/boundaries_png'
drive_urban_boundaries_path = '/content/drive/MyDrive/LoveDA/boundaries/Urban/boundaries_png'

# Ensure the destination directories exist
os.makedirs(drive_rural_boundaries_path, exist_ok=True)
os.makedirs(drive_urban_boundaries_path, exist_ok=True)

# Copy the directories
#shutil.copytree(rural_boundaries_path, drive_rural_boundaries_path, dirs_exist_ok=True)
shutil.copytree(urban_boundaries_path, drive_urban_boundaries_path, dirs_exist_ok=True)

print("Folders have been copied to Google Drive.")

Folders have been copied to Google Drive.


In [6]:
!pwd
!ls

/content
drive  sample_data  Train  Train.zip
